This notebook has to run in an environment with Fastai, on Windows it will require WSL.

In [6]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.notebook import tqdm
import dask.array as da
import pandas as pd
import numpy as np
import pickle as pkl
from tifffile import imread
from skimage.color import gray2rgb

from pathlib import Path
from fastai.learner import load_learner

from sem_quant.processing_utils import extract_regionprops, mask_from_df, suppress_by_iou_mitos, compute_iou_array, add_soma_data
from sem_quant.data_utils import label_func
from sem_quant.load_config import load_config
from sem_quant.utils import smart_path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
config_file_name = "A01_config.json"

In [9]:
# read in config file

config = load_config(config_file_name)

im_path = smart_path(config.paths.im_path)
analysis_dir = smart_path(config.paths.analysis_dir)

output_prefix = config.paths.output_prefix
mitos_data_suffix = config.paths.mitos_data_suffix

axons_res = config.data_properties.axons_res
mitos_res = config.data_properties.mitos_res
row_offset = config.data_properties.row_offset
col_offset = config.data_properties.col_offset

sam2_checkpoint = config.sam_model.sam2_checkpoint
model_cfg = config.sam_model.model_cfg

classifier_path = config.mitos_classifier.path
pad = config.mitos_classifier.pad

min_iou = config.mitos_filters.min_iou
iou_threshold = config.mitos_filters.iou_threshold
min_area = config.mitos_filters.min_area
max_area = config.mitos_filters.max_area
max_eccentricity = config.mitos_filters.max_eccentricity

In [10]:
# details of selecting mito masks
res_adjust = axons_res - mitos_res

properties = ['area', 'area_convex', 'area_filled', 'euler_number', 'image','bbox',
        'eccentricity', 'solidity', 'centroid', 'major_axis_length', 'minor_axis_length']

## Get an image

In [11]:
store = imread(im_path, aszarr=True)
im = da.from_zarr(store, mitos_res)
print('Image shape:', im.shape)

Image shape: (25535, 32073)


## Get main df

In [12]:
df_name = f'{output_prefix}axons.pkl'
df_path = os.path.join(analysis_dir,df_name)
df_cells = pd.read_pickle(df_path)
print('Number of axons:',len(df_cells))
df_cells.head()

Number of axons: 1308


,index,area,bbox,predicted_iou,point_coords,stability_score,crop_box,tile_row_start,tile_row_end,tile_col_start,...,inside_bbox-1,inside_bbox-2,inside_bbox-3,inside_centroid-0,inside_centroid-1,inside_area,inside_eccentricity,inside_major_axis_length,inside_minor_axis_length,inside_label
0,5,4989,"[921.0, 841.0, 95.0, 103.0]",0.969633,"[[1011.96875, 915.71875]]",0.971734,"[639.0, 639.0, 385.0, 385.0]",0,1024,0,...,940.0,924.0,996.0,892.0,968.0,2365.0,0.805341,71.680075,42.492763,1
1,8,6755,"[764.0, 707.0, 97.0, 143.0]",0.966587,"[[771.34375, 795.40625]]",0.963737,"[639.0, 639.0, 385.0, 385.0]",0,1024,0,...,786.0,823.0,840.0,777.0,813.0,3330.0,0.811238,86.215271,50.411460,2
2,15,3100,"[899.0, 916.0, 69.0, 87.0]",0.955799,"[[915.71875, 987.90625]]",0.962575,"[639.0, 639.0, 385.0, 385.0]",0,1024,0,...,914.0,988.0,954.0,964.0,931.0,1404.0,0.670115,51.808427,38.455178,3
3,36,4503,"[112.0, 725.0, 73.0, 110.0]",0.972314,"[[181.40625, 795.40625]]",0.950141,"[0.0, 639.0, 387.0, 385.0]",0,1024,0,...,132.0,817.0,167.0,777.0,149.0,1670.0,0.894216,69.758247,31.226276,4
4,45,3857,"[31.0, 866.0, 73.0, 104.0]",0.962982,"[[84.65625, 891.65625]]",0.964131,"[0.0, 639.0, 387.0, 385.0]",0,1024,0,...,48.0,953.0,88.0,919.0,63.0,1623.0,0.814367,61.739922,35.830757,5


### Get SAM masks

In [13]:
masks_dir = os.path.join(analysis_dir,mitos_data_suffix)
file_list = [x for x in os.listdir(masks_dir) if x.endswith('.pkl')]   
print(f'Found {len(file_list)} mito masks in {masks_dir}')

Found 167 mito masks in /mnt/d/data_analysis/2025_Birder_mito/A_00001_analysis/mitos


In [14]:
df_list = []

for file_name in tqdm(file_list):

    print(f'Processing {file_name}')
    
    # load suggested mito masks
    file_path = os.path.join(masks_dir, file_name)
    masks_org = pkl.load(open(file_path, 'rb'))

    # get info about the cell
    cell_ind = int(file_name.split('_')[1])
    row = df_cells.loc[df_cells['label'] == cell_ind]
    inside_mask = row['inside_image'].values[0]

    # discard mitos not inside axons
    masks_sel = [x for x in masks_org if inside_mask[int(x['point_coords'][0][1]/2**res_adjust), int(x['point_coords'][0][0]/2**res_adjust)]]

    df = pd.DataFrame(masks_sel)
    if len(df) == 0:
        print('No masks found')
        continue

    # get mitos data
    df = df[((df['area'] > min_area) & (df['area'] < max_area) & (df['predicted_iou'] > min_iou))]
    print(f'Number of masks: {len(df)}')

    props_list = [] 
    for ind, row_mito in df.iterrows():
        props = extract_regionprops(row_mito,properties,small_size = min_area)
        # keep origin of the data
        props['origin'] = ind
        props_list.append(props)

    if len(props_list) == 0:
        print('No masks found')
        continue
    
    props_all = pd.concat(props_list, ignore_index=True)

    props_all.columns = [ f'mito_{col}' if col == 'area' else col for col in props_all.columns]

    # Concatenate with the original DataFrame
    df['origin'] = df.index
    df_all = pd.merge(df, props_all,left_on = 'origin', right_on= 'origin', how = 'right').reset_index()

    # drop objects by measured categories
    df_all = df_all[(df_all['eccentricity'] < max_eccentricity)]

    print(f'Number of separate objects: {len(df_all)}')

    # suppress by iou
    df_all = df_all.reset_index(drop=True)
    df_all = suppress_by_iou_mitos(df_all, iou_threshold=iou_threshold)
    df_all = df_all.loc[df_all.keep==1,:]
    print(f'Number of self standing objects: {len(df_all)}')

    # shift into the main coordinate system

    row_start = (int(row['inside_bbox-0'].iloc[0]) + row_offset)*2**res_adjust
    row_end = (int(row['inside_bbox-2'].iloc[0]) + row_offset)*2**res_adjust
    col_start = (int(row['inside_bbox-1'].iloc[0]) + col_offset)*2**res_adjust
    col_end = (int(row['inside_bbox-3'].iloc[0]) + col_offset)*2**res_adjust

    df_all['bbox-0'] = df_all['bbox-0'] + row_start
    df_all['bbox-2'] = df_all['bbox-2'] + row_start
    df_all['bbox-1'] = df_all['bbox-1'] + col_start
    df_all['bbox-3'] = df_all['bbox-3'] + col_start
    df_all['centroid-0'] = df_all['centroid-0'] + row_start
    df_all['centroid-1'] = df_all['centroid-1'] + col_start

    df_all['cell_label'] = cell_ind

    # drop segmentation in the local coord system
    df_all.drop(columns=['segmentation'], inplace=True)
    # drop area of the orignal segmented region
    df_all.drop(columns=['area'], inplace=True)

    df_list.append(df_all)

df_mitos = pd.concat(df_list, ignore_index=True)
df_mitos['label'] = df_mitos.index + 1

  0%|          | 0/167 [00:00<?, ?it/s]

Processing 88EM87A01_000001_mitos.pkl
Number of masks: 9
Number of separate objects: 8
Number of self standing objects: 7
Processing 88EM87A01_000002_mitos.pkl
Number of masks: 13
Number of separate objects: 13
Number of self standing objects: 11
Processing 88EM87A01_000003_mitos.pkl
Number of masks: 9
Number of separate objects: 9
Number of self standing objects: 5
Processing 88EM87A01_000111_mito.pkl
Number of masks: 7
Number of separate objects: 7
Number of self standing objects: 5
Processing 88EM87A01_000112_mito.pkl
Number of masks: 5
Number of separate objects: 5
Number of self standing objects: 4
Processing 88EM87A01_000113_mito.pkl
Number of masks: 0
No masks found
Processing 88EM87A01_000114_mito.pkl
Number of masks: 9
Number of separate objects: 6
Number of self standing objects: 6
Processing 88EM87A01_000115_mito.pkl
Number of masks: 3
Number of separate objects: 2
Number of self standing objects: 2
Processing 88EM87A01_000116_mito.pkl
Number of masks: 7
Number of separate o

## Classify objects

In [15]:
learn = load_learner(classifier_path)
learn.model.cuda()
print(next(learn.model.parameters()).device)  # Should print 'cuda:0'

/home/kasia/miniforge3/envs/fastai-env/lib/python3.11/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


cuda:0


In [16]:

df_mitos['prediction_prob'] = None
df_mitos['prediction_prob'] = df_mitos['prediction_prob'].astype(object)

for ind, row in tqdm(df_mitos.iterrows(), total=len(df_mitos)):

    row_start = int(row['centroid-0'] - pad)
    row_stop = int(row['centroid-0'] + pad)
    col_start = int(row['centroid-1'] - pad)
    col_stop = int(row['centroid-1'] + pad)

    im_mito = gray2rgb(im[row_start:row_stop, col_start:col_stop]).compute()

    cat,_,probs = learn.predict(im_mito)
    df_mitos.loc[ind,'prediction'] = str(cat)
    df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float) 

    df_mitos['prediction_prob'] = df_mitos['prediction_prob'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# save the data frame
df_name = f'{output_prefix}mitos.pkl'
df_mitos.to_pickle(os.path.join(analysis_dir,df_name))

  0%|          | 0/945 [00:00<?, ?it/s]

/tmp/ipykernel_1331657/97888599.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float)


/tmp/ipykernel_1331657/97888599.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float)


/tmp/ipykernel_1331657/97888599.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float)


/tmp/ipykernel_1331657/97888599.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float)


...

/tmp/ipykernel_1331657/97888599.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float)


/tmp/ipykernel_1331657/97888599.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float)


/tmp/ipykernel_1331657/97888599.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float)


/tmp/ipykernel_1331657/97888599.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float)


/tmp/ipykernel_1331657/97888599.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  df_mitos.at[ind,'prediction_prob'] = np.array(probs, dtype=float)
